In [1]:
import netCDF4 as nc
import numpy
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)
import os
import cartopy.feature as cfeature
import glob
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib import cm
#import seaborn as sns
import cartopy.crs as ccrs
import cartopy.feature as cf
import mplotutils as mpu # helper functions for cartopy and matplotlib
#import regionmask
from func_plots import *
from func_stats import *
import geopandas as gpd
import regionmask

import cartopy.crs as ccrs
import cartopy.feature as cf
import mplotutils as mpu # helper functions for cartopy and matplotlib

In [2]:
# Regional plot with coastline and country borders
def format_axes(axes):
    for i, ax in enumerate(axes):
        #ax.set_title(plot_names[i])
        ax.coastlines(resolution='50m', linewidth=0.45)
        ax.add_feature(cf.BORDERS, linewidth=0.3)
        ax.set_extent([lonmin, lonmax, latmin, latmax], crs=data_proj)
        ax.gridlines(draw_labels=False, linewidth=0.3, color="gray", xlocs=range(-180, 180, 10), ylocs=range(-90, 90, 10))

def add_headers(fig, *, row_headers=None, col_headers=None, row_pad=1, col_pad=4, rotate_row_headers=True, **text_kwargs):
    # Based on https://stackoverflow.com/a/25814386
    axes = fig.get_axes()

    for ax in axes:
        sbs = ax.get_subplotspec()

        # Putting headers on cols
        if (col_headers is not None) and sbs.is_first_row():
            ax.annotate(
                col_headers[sbs.colspan.start],
                xy=(0.5, 1),
                xytext=(0, col_pad),
                xycoords="axes fraction",
                textcoords="offset points",
                ha="center",
                va="baseline",
                **text_kwargs,
            )

        # Putting headers on rows
        if (row_headers is not None) and sbs.is_first_col():
            ax.annotate(
                row_headers[sbs.rowspan.start],
                xy=(0, 0.5),
                xytext=(-ax.yaxis.labelpad - row_pad, 0),
                xycoords=ax.transAxes, # ax.yaxis.label replaced by ax.transAxes
                textcoords="offset points",
                ha="center",
                va="center", # vertical even after rotation
                rotation=rotate_row_headers * 90,
                **text_kwargs,
            )

lonmin, lonmax, latmin, latmax = [-11, 37, 35, 70.5]                   # window for plotting
set_lonmin, set_lonmax, set_latmin, set_latmax = [-35, 65, 30, 72.6]   # subset the data to get sensible vmin and vmax for the colorbar
data_proj = ccrs.PlateCarree()
map_proj = ccrs.LambertConformal(central_longitude=15) # for regional maps

In [3]:
file_obj = nc.Dataset('/landclim2/yiyaoy/COSMO-CLM2-simulations/timmean/clm5.0_eur0.5_control_h0_2020-2024.nc_timmean')
lat = file_obj.variables['lat']
lat = np.array(lat)
lon = file_obj.variables['lon']
lon = np.array(lon)
lon[lon>180]=lon[lon>180]-360

In [4]:
# Metzger et al. 2005 regions ("Environmental Stratification of Europe")
# The sub-regional classification is based on Metzger et al. (2005), aggregated as in Jakob et al. (2014) and in the European chapter of IPCC AR5,
# to represent 5 different climate and environmental characteristics: Alpine, Atlantic, Continental, Northern and Southern Europe
# Load regions and combine
gdf = gpd.read_file('/net/exo/landclim/pesieber/data/Metzger-2005_Env-stratification-Europe/enz_v8.shp').set_index('EnZ_name')
regions = gpd.GeoDataFrame({'region': [], 'geometry': []}, crs=gdf.crs)
regions.loc[0] = {'region': 'Alpine', 'geometry': gdf.loc[['ALN','ALS']].unary_union}
regions.loc[1] = {'region': 'Northern', 'geometry': gdf.loc[['BOR','NEM']].unary_union}
regions.loc[2] = {'region': 'Atlantic', 'geometry': gdf.loc[['ATN','ATC','LUS']].unary_union}
regions.loc[3] = {'region': 'Continental', 'geometry': gdf.loc[['CON','PAN']].unary_union}
regions.loc[4] = {'region': 'Southern', 'geometry': gdf.loc[['MDN','MDS','MDM','ANA']].unary_union}
regions = regions.set_crs('EPSG:3035').to_crs('EPSG:4326')

In [5]:
# Generate a 2D or 3D mask that can be applied to xarray data
mask_3D = regionmask.mask_3D_geopandas(regions, lon, lat) # 3D for groupby
mask_3D = mask_3D.assign_coords(region = regions.region.values) # assign names
#mask_3D.to_netcdf(dpath_proc + 'regionmask_3D_Metzger.nc')
#region_dict = regions.region.to_dict()
mask_regions = regionmask.from_geopandas(regions, names="region", abbrevs="_from_name", name="regionmask") # Regions object
mask_2D = regionmask.mask_geopandas(regions, lon, lat) # 2D for plotting
#mask_2D.to_netcdf(dpath_proc + 'regionmask_2D_Metzger.nc')
regions = mask_3D.region.values

In [42]:
# 1) Map each “scenario” name to its file
FILE_PATHS = {
    "Ctl":        "/landclim2/yiyaoy/COSMO-CLM2-simulations/control/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_control_h0_2025-2059.nc_monmean",
    "Aff":     "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_forest/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_forest_h0_2025-2059.nc_monmean",
    "Def":      "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_grass/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_grass_h0_2025-2059.nc_monmean",
    "Brd":      "/landclim2/yiyaoy/COSMO-CLM2-simulations/only_forest_broadleaf/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_only_forest_broadleaf_h0_2025-2059.nc_monmean",
    "AfB":     "/landclim2/yiyaoy/COSMO-CLM2-simulations/all_forest_broadleaf/clm5/post-processing/monmean/mergetime/clm5.0_eur0.5_all_forest_broadleaf_h0_2025-2059.nc_monmean",
}

def _load_and_region_mean(var, path, mask_2D, region_id):
    """Open TSA, mask out >1e9, apply region mask, then lon→lat means."""
    with nc.Dataset(path) as ds:
        arr = ds.variables[var][:]       # → (time, lat, lon)
    arr = np.squeeze(arr)                 # in case there’s a degenerate dim
    arr[arr > 1e9] = np.nan               # drop fill values

    # set everything *outside* this region to nan
    arr[:, mask_2D != region_id] = np.nan

    # first average over lon (axis=2), then over lat (axis=1)
    return np.nanmean(np.nanmean(arr, axis=2), axis=1)


def get_mean_data_for_plot(var, region_id):
    """
    Returns a dict mapping each scenario to its time‐series of region‐mean TSA.
    E.g. out["control"] is the control run’s 1D array (time down to region‐mean).
    """
    return {
        name: _load_and_region_mean(var, path, mask_2D, region_id)
        for name, path in FILE_PATHS.items()
    }

In [ ]:
LWup_M_0 = get_mean_data_for_plot('LWup', 0)
SHF_M_0 = get_mean_data_for_plot('FSH', 0)
LHF_M_0 = get_mean_data_for_plot('EFLX_LH_TOT', 0)
LWdown_M_0 = get_mean_data_for_plot('LWdown', 0)
SWup_M_0 = get_mean_data_for_plot('SWup', 0)
SWdown_M_0 = get_mean_data_for_plot('SWdown', 0)
G_M_0 = get_mean_data_for_plot('EFLX_GNET', 0)

LWup_M_1 = get_mean_data_for_plot('LWup', 1)
SHF_M_1 = get_mean_data_for_plot('FSH', 1)
LHF_M_1 = get_mean_data_for_plot('EFLX_LH_TOT', 1)
LWdown_M_1 = get_mean_data_for_plot('LWdown', 1)
SWup_M_1 = get_mean_data_for_plot('SWup', 1)
SWdown_M_1 = get_mean_data_for_plot('SWdown', 1)
G_M_1 = get_mean_data_for_plot('EFLX_GNET', 1)

LWup_M_2 = get_mean_data_for_plot('LWup', 2)
SHF_M_2 = get_mean_data_for_plot('FSH', 2)
LHF_M_2 = get_mean_data_for_plot('EFLX_LH_TOT', 2)
LWdown_M_2 = get_mean_data_for_plot('LWdown', 2)
SWup_M_2 = get_mean_data_for_plot('SWup', 2)
SWdown_M_2 = get_mean_data_for_plot('SWdown', 2)
G_M_2 = get_mean_data_for_plot('EFLX_GNET', 2)

LWup_M_3 = get_mean_data_for_plot('LWup', 3)
SHF_M_3 = get_mean_data_for_plot('FSH', 3)
LHF_M_3 = get_mean_data_for_plot('EFLX_LH_TOT', 3)
LWdown_M_3 = get_mean_data_for_plot('LWdown', 3)
SWup_M_3 = get_mean_data_for_plot('SWup', 3)
SWdown_M_3 = get_mean_data_for_plot('SWdown', 3)
G_M_3 = get_mean_data_for_plot('EFLX_GNET', 3)

LWup_M_4 = get_mean_data_for_plot('LWup', 4)
SHF_M_4 = get_mean_data_for_plot('FSH', 4)
LHF_M_4 = get_mean_data_for_plot('EFLX_LH_TOT', 4)
LWdown_M_4 = get_mean_data_for_plot('LWdown', 4)
SWup_M_4 = get_mean_data_for_plot('SWup', 4)
SWdown_M_4 = get_mean_data_for_plot('SWdown', 4)
G_M_4 = get_mean_data_for_plot('EFLX_GNET', 4)

In [54]:
SHF_M_0['Ctl'].shape

(420,)

In [55]:
n_months = 12
n_years = 35
def plot_delta_TSA_SHF(var_exp1_control_daymean_SHF, var_exp1_control_daymean_LWup, str_1, var_exp1_control_daymean_SWup, var_exp1_control_daymean_SWdown, var_exp1_control_daymean_LWdown, var_exp1_control_daymean_LHF, var_exp1_control_daymean_Gnet, region_name, ylim1, ylim2):
    var_exp1_control_daymean_SHF_mean = np.mean(var_exp1_control_daymean_SHF.reshape((n_months, n_years), order='F'), axis=1)
    

    var_exp1_control_daymean_LWup_mean = np.mean(var_exp1_control_daymean_LWup.reshape((n_months, n_years), order='F'), axis=1)
    

    var_exp1_control_daymean_SWup_mean = np.mean(var_exp1_control_daymean_SWup.reshape((n_months, n_years), order='F'), axis=1)
    

    var_exp1_control_daymean_SWdown_mean = np.mean(var_exp1_control_daymean_SWdown.reshape((n_months, n_years), order='F'), axis=1)
    

    var_exp1_control_daymean_LWdown_mean = np.mean(var_exp1_control_daymean_LWdown.reshape((n_months, n_years), order='F'), axis=1)
    

    var_exp1_control_daymean_LHF_mean = np.mean(var_exp1_control_daymean_LHF.reshape((n_months, n_years), order='F'), axis=1)
    

    var_exp1_control_daymean_Gnet_mean = np.mean(var_exp1_control_daymean_Gnet.reshape((n_months, n_years), order='F'), axis=1)
    

    plt.plot(X, var_exp1_control_daymean_LWup_mean, color='dodgerblue', label='$\\Delta$$\mathregular{LW_{up,'+str_1+'}}$',marker='o')
    plt.plot(X, var_exp1_control_daymean_SHF_mean, color='tomato', label='$\\Delta$$\mathregular{H_{up,'+str_1+'}}$',marker='o')
    plt.legend(fontsize=12,loc='upper left', framealpha=0.5)
    plt.hlines(y=0, xmin=-1, xmax=12, color='k')
    
    plt.xlim(-1,12)
    # plt.legend(fontsize=12,loc='upper left')
    plt.xticks(X, ['J','F','M','A','M','J','J','A','S','O','N','D'], fontsize=16)
    plt.yticks(fontsize=14)
    
    # plt.xlabel('Alpine', fontsize=16)
    ax1.set_title(region_name, fontsize=20)
    plt.ylim(ylim1,ylim2)
    ax2 = ax1.twinx()

    plt.bar(X-0.3, var_exp1_control_daymean_SWdown_mean-var_exp1_control_daymean_SWup_mean, width=0.2, color='fuchsia', alpha=0.8, label='$\\Delta$$\mathregular{SW_{net,'+str_1+'}}$')
    plt.bar(X-0.1, var_exp1_control_daymean_LWdown_mean, width=0.2, color='lightgreen', alpha=0.8, label='$\\Delta$$\mathregular{LW_{down,'+str_1+'}}$')
    plt.bar(X+0.1, -var_exp1_control_daymean_Gnet_mean, width=0.2, color='sandybrown', alpha=0.8, label='-$\\Delta$$\mathregular{G_{down,'+str_1+'}}$')
    plt.bar(X+0.3, -var_exp1_control_daymean_LHF_mean, width=0.2, color='aqua', alpha=0.8, label='-$\\Delta$$\mathregular{LE_{up,'+str_1+'}}$')
    
    
    
    plt.ylim(ylim1,ylim2)
    plt.yticks(fontsize=14)
    plt.legend(fontsize=12,loc='upper right', framealpha=0.5)

In [67]:
LWup_M_0['Ctl'].reshape((n_months, n_years), order='F').shape

(12, 35)

In [68]:
x1 = np.mean(test[5:8, :], axis=0)

In [69]:
x1.shape

(35,)

In [78]:
import scipy.stats as stats
def get_value(data):
    data_reshape = data.reshape((n_months, n_years), order='F')
    summer_data = data_reshape[5:8, :]
    summer_mean = np.mean(summer_data, axis=0)
    data_mean = np.mean(summer_mean, axis=0)
    std_summer = np.std(summer_mean, ddof=1)
    n = len(summer_mean)
    se = std_summer / np.sqrt(n)
    confidence = 0.95
    t_score = stats.t.ppf((1 + confidence) / 2.0, df=n-1)
    
    return data_mean, t_score*se

In [130]:
get_value(G_M_0['Def'])

(7.1758394, 0.3457568471254527)

In [219]:
data_raw = SWdown_M_4
data = get_value(data_raw['Ctl'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))
data = get_value(data_raw['Brd'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))
data = get_value(data_raw['Def'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))
data = get_value(data_raw['Aff'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))
data = get_value(data_raw['AfB'])
print("{:.2f}".format(data[0]) + "±" + "{:.2f}".format(data[1]))

268.68±1.65
268.07±1.70
269.19±1.68
266.42±1.60
265.85±1.71


In [ ]:

fig, axes = plt.subplots(5,2, figsize=(12,20), subplot_kw=dict(projection=map_proj, facecolor="lightgrey"))
fig.subplots_adjust(hspace=0.3, wspace=0.4, left=0.05, right=0.95, top=0.95, bottom=0.05)

X = np.array(range(12))

ax1 = plt.subplot(521, frameon=True)
plt.ylabel('$\\Delta$Energy flux ($\mathregular{W/m^2}$)', fontsize=16)
plt.title('a', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_0['Brd']-SHF_M_0['Ctl'], LWup_M_0['Brd']-LWup_M_0['Ctl'], 'Brd', SWup_M_0['Brd']-SWup_M_0['Ctl'], SWdown_M_0['Brd']-SWdown_M_0['Ctl'], LWdown_M_0['Brd']-LWdown_M_0['Ctl'], LHF_M_0['Brd']-LHF_M_0['Ctl'], G_M_0['Brd']-G_M_0['Ctl'], 'Alpine', -20, 20)


ax1 = plt.subplot(523, frameon=True)
plt.ylabel('$\\Delta$Energy flux ($\mathregular{W/m^2}$)', fontsize=16)
plt.title('c', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_1['Brd']-SHF_M_1['Ctl'], LWup_M_1['Brd']-LWup_M_1['Ctl'], 'Brd', SWup_M_1['Brd']-SWup_M_1['Ctl'], SWdown_M_1['Brd']-SWdown_M_1['Ctl'], LWdown_M_1['Brd']-LWdown_M_1['Ctl'], LHF_M_1['Brd']-LHF_M_1['Ctl'], G_M_1['Brd']-G_M_1['Ctl'], 'Northern', -30, 30)

ax1 = plt.subplot(525, frameon=True)
plt.ylabel('$\\Delta$Energy flux ($\mathregular{W/m^2}$)', fontsize=16)
plt.title('e', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_2['Brd']-SHF_M_2['Ctl'], LWup_M_2['Brd']-LWup_M_2['Ctl'], 'Brd', SWup_M_2['Brd']-SWup_M_2['Ctl'], SWdown_M_2['Brd']-SWdown_M_2['Ctl'], LWdown_M_2['Brd']-LWdown_M_2['Ctl'], LHF_M_2['Brd']-LHF_M_2['Ctl'], G_M_2['Brd']-G_M_2['Ctl'], 'Atlantic', -10, 10)

ax1 = plt.subplot(527, frameon=True)
plt.ylabel('$\\Delta$Energy flux ($\mathregular{W/m^2}$)', fontsize=16)
plt.title('g', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_3['Brd']-SHF_M_3['Ctl'], LWup_M_3['Brd']-LWup_M_3['Ctl'], 'Brd', SWup_M_3['Brd']-SWup_M_3['Ctl'], SWdown_M_3['Brd']-SWdown_M_3['Ctl'], LWdown_M_3['Brd']-LWdown_M_3['Ctl'], LHF_M_3['Brd']-LHF_M_3['Ctl'], G_M_3['Brd']-G_M_3['Ctl'], 'Continental', -10, 10)

ax1 = plt.subplot(529, frameon=True)
plt.ylabel('$\\Delta$Energy flux ($\mathregular{W/m^2}$)', fontsize=16)
plt.title('i', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_4['Brd']-SHF_M_4['Ctl'], LWup_M_4['Brd']-LWup_M_4['Ctl'], 'Brd', SWup_M_4['Brd']-SWup_M_4['Ctl'], SWdown_M_4['Brd']-SWdown_M_4['Ctl'], LWdown_M_4['Brd']-LWdown_M_4['Ctl'], LHF_M_4['Brd']-LHF_M_4['Ctl'], G_M_4['Brd']-G_M_4['Ctl'], 'Southern', -10, 10)



ax1 = plt.subplot(522, frameon=True)
plt.title('b', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_0['Def']-SHF_M_0['Ctl'], LWup_M_0['Def']-LWup_M_0['Ctl'], 'Def', SWup_M_0['Def']-SWup_M_0['Ctl'], SWdown_M_0['Def']-SWdown_M_0['Ctl'], LWdown_M_0['Def']-LWdown_M_0['Ctl'], LHF_M_0['Def']-LHF_M_0['Ctl'], G_M_0['Def']-G_M_0['Ctl'], 'Alpine', -20, 20)


ax1 = plt.subplot(524, frameon=True)
plt.title('d', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_1['Def']-SHF_M_1['Ctl'], LWup_M_1['Def']-LWup_M_1['Ctl'], 'Def', SWup_M_1['Def']-SWup_M_1['Ctl'], SWdown_M_1['Def']-SWdown_M_1['Ctl'], LWdown_M_1['Def']-LWdown_M_1['Ctl'], LHF_M_1['Def']-LHF_M_1['Ctl'], G_M_1['Def']-G_M_1['Ctl'], 'Northern', -30, 30)

ax1 = plt.subplot(526, frameon=True)
plt.title('f', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_2['Def']-SHF_M_2['Ctl'], LWup_M_2['Def']-LWup_M_2['Ctl'], 'Def', SWup_M_2['Def']-SWup_M_2['Ctl'], SWdown_M_2['Def']-SWdown_M_2['Ctl'], LWdown_M_2['Def']-LWdown_M_2['Ctl'], LHF_M_2['Def']-LHF_M_2['Ctl'], G_M_2['Def']-G_M_2['Ctl'], 'Atlantic', -10, 10)

ax1 = plt.subplot(528, frameon=True)
plt.title('h', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_3['Def']-SHF_M_3['Ctl'], LWup_M_3['Def']-LWup_M_3['Ctl'], 'Def', SWup_M_3['Def']-SWup_M_3['Ctl'], SWdown_M_3['Def']-SWdown_M_3['Ctl'], LWdown_M_3['Def']-LWdown_M_3['Ctl'], LHF_M_3['Def']-LHF_M_3['Ctl'], G_M_3['Def']-G_M_3['Ctl'], 'Continental', -10, 10)

ax1 = plt.subplot(5,2,10, frameon=True)
plt.title('j', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_4['Def']-SHF_M_4['Ctl'], LWup_M_4['Def']-LWup_M_4['Ctl'], 'Def', SWup_M_4['Def']-SWup_M_4['Ctl'], SWdown_M_4['Def']-SWdown_M_4['Ctl'], LWdown_M_4['Def']-LWdown_M_4['Ctl'], LHF_M_4['Def']-LHF_M_4['Ctl'], G_M_4['Def']-G_M_4['Ctl'], 'Southern', -10, 10)


In [ ]:

fig, axes = plt.subplots(5,2, figsize=(12,20), subplot_kw=dict(projection=map_proj, facecolor="lightgrey"))
fig.subplots_adjust(hspace=0.3, wspace=0.4, left=0.05, right=0.95, top=0.95, bottom=0.05)

X = np.array(range(12))

ax1 = plt.subplot(521, frameon=True)
plt.ylabel('$\\Delta$Energy flux ($\mathregular{W/m^2}$)', fontsize=16)
plt.title('a', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_0['Aff']-SHF_M_0['Ctl'], LWup_M_0['Aff']-LWup_M_0['Ctl'], 'Aff', SWup_M_0['Aff']-SWup_M_0['Ctl'], SWdown_M_0['Aff']-SWdown_M_0['Ctl'], LWdown_M_0['Aff']-LWdown_M_0['Ctl'], LHF_M_0['Aff']-LHF_M_0['Ctl'], G_M_0['Aff']-G_M_0['Ctl'], 'Alpine', -20, 20)


ax1 = plt.subplot(523, frameon=True)
plt.ylabel('$\\Delta$Energy flux ($\mathregular{W/m^2}$)', fontsize=16)
plt.title('c', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_1['Aff']-SHF_M_1['Ctl'], LWup_M_1['Aff']-LWup_M_1['Ctl'], 'Aff', SWup_M_1['Aff']-SWup_M_1['Ctl'], SWdown_M_1['Aff']-SWdown_M_1['Ctl'], LWdown_M_1['Aff']-LWdown_M_1['Ctl'], LHF_M_1['Aff']-LHF_M_1['Ctl'], G_M_1['Aff']-G_M_1['Ctl'], 'Northern', -30, 30)

ax1 = plt.subplot(525, frameon=True)
plt.ylabel('$\\Delta$Energy flux ($\mathregular{W/m^2}$)', fontsize=16)
plt.title('e', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_2['Aff']-SHF_M_2['Ctl'], LWup_M_2['Aff']-LWup_M_2['Ctl'], 'Aff', SWup_M_2['Aff']-SWup_M_2['Ctl'], SWdown_M_2['Aff']-SWdown_M_2['Ctl'], LWdown_M_2['Aff']-LWdown_M_2['Ctl'], LHF_M_2['Aff']-LHF_M_2['Ctl'], G_M_2['Aff']-G_M_2['Ctl'], 'Atlantic', -10, 10)

ax1 = plt.subplot(527, frameon=True)
plt.ylabel('$\\Delta$Energy flux ($\mathregular{W/m^2}$)', fontsize=16)
plt.title('g', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_3['Aff']-SHF_M_3['Ctl'], LWup_M_3['Aff']-LWup_M_3['Ctl'], 'Aff', SWup_M_3['Aff']-SWup_M_3['Ctl'], SWdown_M_3['Aff']-SWdown_M_3['Ctl'], LWdown_M_3['Aff']-LWdown_M_3['Ctl'], LHF_M_3['Aff']-LHF_M_3['Ctl'], G_M_3['Aff']-G_M_3['Ctl'], 'Continental', -10, 10)

ax1 = plt.subplot(529, frameon=True)
plt.ylabel('$\\Delta$Energy flux ($\mathregular{W/m^2}$)', fontsize=16)
plt.title('i', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_4['Aff']-SHF_M_4['Ctl'], LWup_M_4['Aff']-LWup_M_4['Ctl'], 'Aff', SWup_M_4['Aff']-SWup_M_4['Ctl'], SWdown_M_4['Aff']-SWdown_M_4['Ctl'], LWdown_M_4['Aff']-LWdown_M_4['Ctl'], LHF_M_4['Aff']-LHF_M_4['Ctl'], G_M_4['Aff']-G_M_4['Ctl'], 'Southern', -10, 10)



ax1 = plt.subplot(522, frameon=True)
plt.title('b', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_0['AfB']-SHF_M_0['Ctl'], LWup_M_0['AfB']-LWup_M_0['Ctl'], 'AfB', SWup_M_0['AfB']-SWup_M_0['Ctl'], SWdown_M_0['AfB']-SWdown_M_0['Ctl'], LWdown_M_0['AfB']-LWdown_M_0['Ctl'], LHF_M_0['AfB']-LHF_M_0['Ctl'], G_M_0['AfB']-G_M_0['Ctl'], 'Alpine', -20, 20)


ax1 = plt.subplot(524, frameon=True)
plt.title('d', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_1['AfB']-SHF_M_1['Ctl'], LWup_M_1['AfB']-LWup_M_1['Ctl'], 'AfB', SWup_M_1['AfB']-SWup_M_1['Ctl'], SWdown_M_1['AfB']-SWdown_M_1['Ctl'], LWdown_M_1['AfB']-LWdown_M_1['Ctl'], LHF_M_1['AfB']-LHF_M_1['Ctl'], G_M_1['AfB']-G_M_1['Ctl'], 'Northern', -30, 30)

ax1 = plt.subplot(526, frameon=True)
plt.title('f', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_2['AfB']-SHF_M_2['Ctl'], LWup_M_2['AfB']-LWup_M_2['Ctl'], 'AfB', SWup_M_2['AfB']-SWup_M_2['Ctl'], SWdown_M_2['AfB']-SWdown_M_2['Ctl'], LWdown_M_2['AfB']-LWdown_M_2['Ctl'], LHF_M_2['AfB']-LHF_M_2['Ctl'], G_M_2['AfB']-G_M_2['Ctl'], 'Atlantic', -10, 10)

ax1 = plt.subplot(528, frameon=True)
plt.title('h', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_3['AfB']-SHF_M_3['Ctl'], LWup_M_3['AfB']-LWup_M_3['Ctl'], 'AfB', SWup_M_3['AfB']-SWup_M_3['Ctl'], SWdown_M_3['AfB']-SWdown_M_3['Ctl'], LWdown_M_3['AfB']-LWdown_M_3['Ctl'], LHF_M_3['AfB']-LHF_M_3['Ctl'], G_M_3['AfB']-G_M_3['Ctl'], 'Continental', -10, 10)

ax1 = plt.subplot(5,2,10, frameon=True)
plt.title('j', fontsize=14, loc='left', weight='bold')
plot_delta_TSA_SHF(SHF_M_4['AfB']-SHF_M_4['Ctl'], LWup_M_4['AfB']-LWup_M_4['Ctl'], 'AfB', SWup_M_4['AfB']-SWup_M_4['Ctl'], SWdown_M_4['AfB']-SWdown_M_4['Ctl'], LWdown_M_4['AfB']-LWdown_M_4['Ctl'], LHF_M_4['AfB']-LHF_M_4['Ctl'], G_M_4['AfB']-G_M_4['Ctl'], 'Southern', -10, 10)
